In [97]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from pydub import AudioSegment as AS
from pydub.playback import play
import os
import json
import csv

In [103]:
#prepare dataset
def chop_song(songfile, chop):
    song = AS.from_mp3(songfile)
    song_array = []
    for i in range(0, int(len(song)/(chop * 1000))):
        raw = song[i*1000*chop:(i+1)*1000*chop].raw_data
        song_array.append(list(raw))
    return song_array
path = "../low_data/"
files = []
for r, d, f in os.walk(path):
    paths = [r + "/" + i for i in f if i.endswith(".mp3")]
    files += paths
songs_to_label = {}
label_to_name = {} #human readability
with open("../metadata/raw_tracks.csv") as f:
    data = csv.DictReader(f)
    for d in data:
        try:
            genre = json.loads(d["track_genres"].replace("\'", "\""))[0]
            songs_to_label[d['track_id']] = genre['genre_id']
            human_to_name[genre['genre_id']] = genre['genre_title']
        except:
            pass

songs = []
for count, f in enumerate(files):
    songs += chop_song(f, 3)
    if count > 3:
        break
print(len(songs))
print(len(songs[3]))

46
529200
